# Graphs in Raku

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Day 12 of Raku Advent 2024 </span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
December 2024

-----

## Introduction

This blog post discusses the development of graph theory algorithms in Raku.

In this document:
- All computational graph features discussed are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).   
- Graph plotting is done with:
    - `js-d3-graph-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
    -  `Graph.dot`, that makes SVG images via [Graphviz](https://graphviz.org).

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [2]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::HexagonalGrid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Nested;
use Graph::Path;
use Graph::Petersen;
use Graph::Star;
use Graph::TriangularGrid;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Text::Levenshtein::Damerau;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;
use Graphviz::DOT::Grammar;
use Graphviz::DOT::Chessboard;

use JavaScript::D3;
use WWW::MermaidInk;
use Data::Importers;
use Markdown::Grammar;
use LLM::Configurations;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [62]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [63]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [64]:
my $background = '#1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $stroke-color = 'SlateGray';
my $tick-labels-color = 'Silver';
my $tick-labels-font-family = 'Helvetica';
my $tick-labels-font-size = 10;
my $title-color = 'Ivory';
my $tooltip-background-color = 'none';
my $tooltip-color = 'LightBlue';
my $vertex-size = 6;
my $engine = 'neato';
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => #1F1F1F, edge-thickness => 3, title-color => Ivory, vertex-size => 6}

----- 

## Graph theory functionalities making

Because of a few "serious" projects involving conversational agents about logistics I thought that is a good idea to have geographical shortest path finding in Raku. (Instead of "outsourcing" those computations to some other system.) 

The assumption were that:
- The fundamental graph algorithms are (relatively) easy to program.
- Just good core data structures have to be chosen 
- Data and algorithms for [simple graphs](https://mathworld.wolfram.com/SimpleGraph.html) should be easy to implement.

The above assumptions turned true, in general, and while graph-programming with Raku, in particular. 

The Raku package ["Graph"](https://raku.land/zef:antononcube/Graph) provides the class `Graph` that has the following classes of algorithms (as class methods):

- Finding paths cycles and flows
    - Finding shortest path, hamiltonian path, etc.
- Matching and coloring
- Graph manipulation, 
    - Edge removal, vertex replacement, etc.
- Graph-to-graph combinations
    - Union, difference, etc.
- Construction of parameterized graphs
    - Complete, star, wheel, etc.
- Construction of model distribution based random graphs
    - Watts-Strogatz, Barabási–Albert, etc.

The class `Graph` uses a map of maps for the adjacency lists (i.e. vertex connections.) It _should be_ extended to have vertex- and edge tags. Having multiple edges should be supported at some point, but I consider it low priority. 

Now, [Graph theory](https://en.wikipedia.org/wiki/Graph_theory) is a huge field and this inevitably produces a certain opinionated selection of which algorithms to implement first or implement at all.
It is natural to consider having (1) fundamental graph algorithms implemented and (2) a framework for quicker developing of new ones. Currently, I would say, the former fairly well addressed. It is in my future plans to have user-exposed framework implementation based on Depth-First Search (DFS) and Breath-First Search (BFS) algorithms.

Graphs are naturally related to sparse matrices and many graph algorithms can be recast into sparse matrix algebra based algorithms. Because of this relationship _and_ because sparse matrices are a very useful mathematical tool I implemented the packages 
["Math::SparseMatrix"](https://raku.land/zef:antononcube/Math::SparseMatrix) and 
["Math::SparseMatrix::Native"](https://raku.land/zef:antononcube/Math::SparseMatrix::Native).

The current functionalities of `Graph` are demonstrated and explained in the "neat examples" videos and blog posts. Here is the videos list in order of their making:

- ["Graph neat examples in Raku (Set 1)"](https://youtu.be/5qXgqqRZHow)
- ["Sparse matrix neat examples in Raku"](https://youtu.be/kQo3wpiUu6w)
- ["Graph neat examples in Raku (Set 2)"](https://youtu.be/E7qhutQcWCY)
- ["Graph neat examples in Raku (Set 3)"](https://youtu.be/S_3e7liz4KM)
- ["Chess positions and knight's tours via graphs (in Raku)"](https://youtu.be/fwQrQyWC7R0)


-----

## Spanning tree for large cities of USA

In relation to the Geo-motivation projects mentioned above let us consider the problem of building an interstate highway system joining the cities of USA.

Here is the available Geographical data from ["Data::Geographics"](https://raku.land/zef:antononcube/Data::Geographics):

In [65]:
city-data().map(*<Country>).unique.sort

(Botswana Bulgaria Canada Germany Hungary Russia Spain Ukraine United States)

Here is a dataset of large enough cities:

In [90]:
my $maxPop = 200_000;
my @cities = city-data().grep({ $_<Country> eq 'United States' && $_<Population> ≥ $maxPop });
@cities.elems

126

**Remark:** Geographical data and geo-distance computation is provided by ["Data::Geographics"](https://raku.land/zef:antononcube/Data::Geographics).

In [91]:
#% html
@cities.head(4) 
==> to-html(field-names => <ID Country State City Latitude Longitude Elevation LocationLink>)

ID,Country,State,City,Latitude,Longitude,Elevation,LocationLink
United_States.New_York.New_York_City,United States,New York,New York City,40.6642738,-73.9385004,10,"http://maps.google.com/maps?q=40.6642738,-73.9385004&z=12&t=h"
United_States.California.Los_Angeles,United States,California,Los Angeles,34.0193936,-118.4108248,89,"http://maps.google.com/maps?q=34.0193936,-118.4108248&z=12&t=h"
United_States.Illinois.Chicago,United States,Illinois,Chicago,41.8375511,-87.6818441,179,"http://maps.google.com/maps?q=41.8375511,-87.6818441&z=12&t=h"
United_States.Texas.Houston,United States,Texas,Houston,29.7804724,-95.3863425,12,"http://maps.google.com/maps?q=29.7804724,-95.3863425&z=12&t=h"


Here is the corresponding weighted **complete** graph:

In [92]:
my %coords = @cities.map({ $_<City> => ($_<Latitude>, $_<Longitude>) });
%coords .= grep({ $_.key ∉ <Anchorage Honolulu> });
my @dsEdges = (%coords.keys X %coords.keys ).map({ %(from => $_.head, to => $_.tail, weight => geo-distance(|%coords{$_.head}, |%coords{$_.tail} )) });
my $gGeo = Graph.new(@dsEdges, :!directed)

Graph(vertexes => 122, edges => 7503, directed => False)

Here is the corresponding spanning tree:

In [93]:
my $stree = $gGeo.find-spanning-tree(method => 'kruskal')

Graph(vertexes => 122, edges => 121, directed => False)

And here is the plot of that tree:

In [94]:
#% js
$stree.edges
==> js-d3-graph-plot(
    vertex-coordinates => %coords.nodemap(*.reverse)».List.Hash,
    title => "USA, cities with population ≥ $maxPop",
    width => 1400,
    height => 700,
    :$background, :$title-color, :$edge-thickness, 
    vertex-size => 4,
    vertex-label-font-size => 12,
    vertex-label-font-family => Whatever,
    margins => {right => 200}
    )

Here is an example of finding the shortest path from Seattle to Jacksonville in the spanning tree.

In [95]:
$stree.find-shortest-path('Seattle', 'Jacksonville')

[Seattle Spokane Boise City Reno Sacramento Stockton Modesto Fresno Bakersfield Santa Clarita Los Angeles Long Beach Anaheim Santa Ana Irvine Riverside Fontana San Bernardino Enterprise Henderson Glendale Phoenix Scottsdale Mesa Gilbert Tucson El Paso Albuquerque Amarillo Oklahoma City Tulsa Little Rock Memphis Nashville Huntsville Birmingham Montgomery Columbus Jacksonville]

-----

## Visualization

Graph visualization is extremely important for development of graph software. (Meaning, it speeds up the development tremendously.)
Immediately after I implemented the first few "main" algorithms I made the corresponding Wolfram Language (WL) and Mermaid-JS representations methods of `Graph`. 
Soon after, I implemented `js-d3-graph-plot` in ["JavaScript:D3"](https://raku.land/zef:antononcube/JavaScript::D3) -- [D3.js](https://d3js.org) has a very nice [_network force_](https://d3js.org/d3-force) functionalities. ("Network" is the D3.js lingo for "graph.") Since, the using of JavaScript plots rendering in Jupyter notebooks is somewhat capricious, I looked for alternatives. This lead to a "full blown" [Graphviz](https://graphviz.org) plot support in `Graph`. (More about that below.)

### JavaScript visualization

The plotting with [D3.js](https://d3js.org) (via ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3)) made me very enthusiastic to implement- and curious to see how different named and/or parameterized graphs would look visualized. So, I quickly made half a dozen of them. These graphs are have known properties, which makes them good for algorithm development testing. 

Here is a map (dictionary) with most of the currently implemented parameterized graphs:

In [5]:
my %namedGraphs = 
    Circulant => Graph::Circulant.new(7, 3),
    Complete => Graph::Complete.new(5),
    CompleteKaryTree => Graph::CompleteKaryTree.new(3,3),
    Cycle => Graph::Cycle.new(8),
    Grid => Graph::Grid.new(4,3),
    TriangularGrid => Graph::TriangularGrid.new(3,3),
    HexagonalGrid => Graph::HexagonalGrid.new(2,2),
    Hypercube => Graph::Hypercube.new(4),
    KnightTour => Graph::KnightTour.new(6,4),
    Path => Graph::Path.new('a'..'g', :directed),
    Petersen => Graph::Petersen.new(),
    Star => Graph::Star.new(5),
    Wheel => Graph::Wheel.new(7, :!directed);

.say for %namedGraphs

Circulant => Graph(vertexes => 7, edges => 7, directed => False)
Complete => Graph(vertexes => 5, edges => 10, directed => False)
Hypercube => Graph(vertexes => 16, edges => 32, directed => False)
Cycle => Graph(vertexes => 8, edges => 8, directed => False)
TriangularGrid => Graph(vertexes => 16, edges => 33, directed => False)
Wheel => Graph(vertexes => 8, edges => 14, directed => False)
CompleteKaryTree => Graph(vertexes => 13, edges => 12, directed => False)
HexagonalGrid => Graph(vertexes => 16, edges => 19, directed => False)
Petersen => Graph(vertexes => 10, edges => 15, directed => False)
KnightTour => Graph(vertexes => 24, edges => 44, directed => False)
Path => Graph(vertexes => 7, edges => 6, directed => True)
Grid => Graph(vertexes => 12, edges => 17, directed => False)
Star => Graph(vertexes => 6, edges => 5, directed => False)


In [6]:
#%js
%namedGraphs.pairs.sort(*.key).map({ 
    js-d3-graph-plot(
        $_.value.edges(:dataset),
        vertex-coordinates => $_.value.vertex-coordinates,
        :$background, :$title-color, :$edge-thickness, :$vertex-size,
        directed => $_.value.directed,
        title => $_.key, 
        width => 300,
        height => 300, 
        force => {charge => {strength => -200}, link => {minDistance => 40}}
    )
 }).join("\n")

Of particular interest to me are the random graphs:
- Simple vertex- and edge sampling random graphs are useful for testing
- Well known random graphs with adhering to particular models are also very nice to have

The easiest way to have random graphs implementations is to have a special `Graph::Distribution` class. 

Here are examples:

In [97]:
my %randomGraphs = 
    Barabasi-Albert => Graph::Random.new(Graph::Distribution::BarabasiAlbert.new(20,2)),
    "Price's model" => Graph::Random.new(Graph::Distribution::Price.new(14, 2, 1)),
    Random => Graph::Random.new(13,16),
    Watts-Strogatz => Graph::Random.new(Graph::Distribution::WattsStrogatz.new(20,0.07));

.say for %randomGraphs

Random => Graph(vertexes => 13, edges => 16, directed => False)
Barabasi-Albert => Graph(vertexes => 20, edges => 35, directed => False)
Watts-Strogatz => Graph(vertexes => 20, edges => 43, directed => False)
Price's model => Graph(vertexes => 14, edges => 23, directed => True)


In [98]:
#%js
%randomGraphs.pairs.sort(*.key).map({ 
    js-d3-graph-plot(
        $_.value.edges(:dataset),
        :$background, :$title-color, :$edge-thickness, :$vertex-size,
        directed => $_.value.directed,
        title => $_.key, 
        width => 400, 
        force => {charge => {strength => -200}, link => {minDistance => 40}}
    )
 }).join("\n")

### Graphviz visualization

`Graph` has the method `dot` that translate the graph object into a [Graphviz DOT language](https://graphviz.org/doc/info/lang.html) spec.

In [99]:
#% html
%randomGraphs<Watts-Strogatz>.dot(:$background, engine => 'sfdp', :svg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 18 -->
 
 18 
 
 18 
 
<!-- 0--18 -->
 
 0--18 
 
 
<!-- 19 -->
 
 19 
 
 19 
 
<!-- 0--19 -->
 
 0--19 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 1--19 -->
 
 1--19 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 1--3 -->
 
 1--3 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 10--12 -->
 
 10--12 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 10--8 -->
 
 10--8 
 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 11--13 -->
 
 11--13 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 11--6 -->
 
 11--6 
 
 
<!-- 11--9 -->
 
 11--9 
 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 12--14 -->
 
 12--14 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 12--15 -->
 
 12--15 
 
 
<!-- 13--14 -->
 
 13--14 
 
 
<!-- 13--15 -->
 
 13--15 
 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 14--16 -->
 
 14--16 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 17 -->
 
 17 
 
 17 
 
<!-- 15--17 -->
 
 15--17 
 
 
<!-- 16--17 -->
 
 16--17 
 
 
<!-- 16--18 -->
 
 16--18 
 
 
<!-- 17--18 -->
 
 17--18 
 
 
<!-- 17--19 -->
 
 17--19 
 
 
<!-- 18--19 -->
 
 18--19 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 2--6 -->
 
 2--6 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 3--5 -->
 
 3--5 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 4--6 -->
 
 4--6 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 5--7 -->
 
 5--7 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 6--8 -->
 
 6--8 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 7--9 -->
 
 7--9 
 
 
<!-- 8--9 -->
 
 8--9

Using the "neato" layout engine graph objects that have vertex coordinates are properly plotted:

In [100]:
#% html
Graph::KnightTour.new(4, 6).dot(:$background, engine => 'neato', :svg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0_1 -->
 
 0_1 
 
 0_1 
 
<!-- 1_3 -->
 
 1_3 
 
 1_3 
 
<!-- 0_1--1_3 -->
 
 0_1--1_3 
 
 
<!-- 2_2 -->
 
 2_2 
 
 2_2 
 
<!-- 0_1--2_2 -->
 
 0_1--2_2 
 
 
<!-- 2_0 -->
 
 2_0 
 
 2_0 
 
<!-- 0_1--2_0 -->
 
 0_1--2_0 
 
 
<!-- 0_0 -->
 
 0_0 
 
 0_0 
 
<!-- 1_2 -->
 
 1_2 
 
 1_2 
 
<!-- 0_0--1_2 -->
 
 0_0--1_2 
 
 
<!-- 2_1 -->
 
 2_1 
 
 2_1 
 
<!-- 0_0--2_1 -->
 
 0_0--2_1 
 
 
<!-- 4_3 -->
 
 4_3 
 
 4_3 
 
<!-- 5_1 -->
 
 5_1 
 
 5_1 
 
<!-- 4_3--5_1 -->
 
 4_3--5_1 
 
 
<!-- 3_3 -->
 
 3_3 
 
 3_3 
 
<!-- 1_2--3_3 -->
 
 1_2--3_3 
 
 
<!-- 3_1 -->
 
 3_1 
 
 3_1 
 
<!-- 1_2--3_1 -->
 
 1_2--3_1 
 
 
<!-- 1_2--2_0 -->
 
 1_2--2_0 
 
 
<!-- 4_2 -->
 
 4_2 
 
 4_2 
 
<!-- 5_0 -->
 
 5_0 
 
 5_0 
 
<!-- 4_2--5_0 -->
 
 4_2--5_0 
 
 
<!-- 5_2 -->
 
 5_2 
 
 5_2 
 
<!-- 1_0 -->
 
 1_0 
 
 1_0 
 
<!-- 1_0--3_1 -->
 
 1_0--3_1 
 
 
<!-- 1_0--2_2 -->
 
 1_0--2_2 
 
 
<!-- 3_3--5_2 -->
 
 3_3--5_2 
 
 
<!-- 4_1 -->
 
 4_1 
 
 4_1 
 
<!-- 3_3--4_1 -->
 
 3_3--4_1 
 
 
<!-- 1_3--2_1 -->
 
 1_3--2_1 
 
 
<!-- 3_2 -->
 
 3_2 
 
 3_2 
 
<!-- 1_3--3_2 -->
 
 1_3--3_2 
 
 
<!-- 3_0 -->
 
 3_0 
 
 3_0 
 
<!-- 3_0--4_2 -->
 
 3_0--4_2 
 
 
<!-- 3_0--5_1 -->
 
 3_0--5_1 
 
 
<!-- 5_3 -->
 
 5_3 
 
 5_3 
 
<!-- 4_1--5_3 -->
 
 4_1--5_3 
 
 
<!-- 0_2 -->
 
 0_2 
 
 0_2 
 
<!-- 0_2--1_0 -->
 
 0_2--1_0 
 
 
<!-- 2_3 -->
 
 2_3 
 
 2_3 
 
<!-- 0_2--2_3 -->
 
 0_2--2_3 
 
 
<!-- 0_2--2_1 -->
 
 0_2--2_1 
 
 
<!-- 2_3--4_2 -->
 
 2_3--4_2 
 
 
<!-- 2_3--3_1 -->
 
 2_3--3_1 
 
 
<!-- 3_1--4_3 -->
 
 3_1--4_3 
 
 
<!-- 3_1--5_2 -->
 
 3_1--5_2 
 
 
<!-- 3_1--5_0 -->
 
 3_1--5_0 
 
 
<!-- 2_2--4_3 -->
 
 2_2--4_3 
 
 
<!-- 2_2--3_0 -->
 
 2_2--3_0 
 
 
<!-- 2_2--4_1 -->
 
 2_2--4_1 
 
 
<!-- 2_1--4_2 -->
 
 2_1--4_2 
 
 
<!-- 2_1--3_3 -->
 
 2_1--3_3 
 
 
<!-- 4_0 -->
 
 4_0 
 
 4_0 
 
<!-- 2_1--4_0 -->
 
 2_1--4_0 
 
 
<!-- 1_1 -->
 
 1_1 
 
 1_1 
 
<!-- 1_1--3_0 -->
 
 1_1--3_0 
 
 
<!-- 1_1--2_3 -->
 
 1_1--2_3 
 
 
<!-- 1_1--3_2 -->
 
 1_1--3_2 
 
 
<!-- 2_0--4_1 -->
 
 2_0--4_1 
 
 
<!-- 2_0--3_2 -->
 
 2_0--3_2 
 
 
<!-- 0_3 -->
 
 0_3 
 
 0_3 
 
<!-- 0_3--2_2 -->
 
 0_3--2_2 
 
 
<!-- 0_3--1_1 -->
 
 0_3--1_1 
 
 
<!-- 4_0--5_2 -->
 
 4_0--5_2 
 
 
<!-- 3_2--5_1 -->
 
 3_2--5_1 
 
 
<!-- 3_2--4_0 -->
 
 3_2--4_0 
 
 
<!-- 3_2--5_3 -->
 
 3_2--5_3

Also the Graphviz DOT representations can be used to plot chessboards:

In [6]:
#% html
dot-chessboard(:4r, :6c):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- cb-square-0_5 -->
 
 cb-square-0_5 
 
 
<!-- cb-square-1_5 -->
 
 cb-square-1_5 
 
 
<!-- cb-square-0_5--cb-square-1_5 -->
<!-- cb-square-1_3 -->
 
 cb-square-1_3 
 
 
<!-- cb-square-1_4 -->
 
 cb-square-1_4 
 
 
<!-- cb-square-1_3--cb-square-1_4 -->
<!-- cb-square-2_3 -->
 
 cb-square-2_3 
 
 
<!-- cb-square-1_3--cb-square-2_3 -->
<!-- cb-square-3_1 -->
 
 cb-square-3_1 
 
 
<!-- cb-square-3_2 -->
 
 cb-square-3_2 
 
 
<!-- cb-square-3_1--cb-square-3_2 -->
<!-- cb-square-3_0 -->
 
 cb-square-3_0 
 
 
<!-- cb-square-3_0--cb-square-3_1 -->
<!-- cb-square-0_4 -->
 
 cb-square-0_4 
 
 
<!-- cb-square-0_4--cb-square-0_5 -->
<!-- cb-square-0_4--cb-square-1_4 -->
<!-- cb-square-3_5 -->
 
 cb-square-3_5 
 
 
<!-- cb-square-2_4 -->
 
 cb-square-2_4 
 
 
<!-- cb-square-1_4--cb-square-2_4 -->
<!-- cb-square-1_4--cb-square-1_5 -->
<!-- cb-square-0_1 -->
 
 cb-square-0_1 
 
 
<!-- cb-square-1_1 -->
 
 cb-square-1_1 
 
 
<!-- cb-square-0_1--cb-square-1_1 -->
<!-- cb-square-0_2 -->
 
 cb-square-0_2 
 
 
<!-- cb-square-0_1--cb-square-0_2 -->
<!-- cb-square-3_4 -->
 
 cb-square-3_4 
 
 
<!-- cb-square-3_4--cb-square-3_5 -->
<!-- cb-square-2_1 -->
 
 cb-square-2_1 
 
 
<!-- cb-square-2_1--cb-square-3_1 -->
<!-- cb-square-2_2 -->
 
 cb-square-2_2 
 
 
<!-- cb-square-2_1--cb-square-2_2 -->
<!-- cb-square-2_4--cb-square-3_4 -->
<!-- cb-square-2_5 -->
 
 cb-square-2_5 
 
 
<!-- cb-square-2_4--cb-square-2_5 -->
<!-- cb-square-2_3--cb-square-2_4 -->
<!-- cb-square-3_3 -->
 
 cb-square-3_3 
 
 
<!-- cb-square-2_3--cb-square-3_3 -->
<!-- cb-square-2_2--cb-square-2_3 -->
<!-- cb-square-2_2--cb-square-3_2 -->
<!-- cb-square-2_5--cb-square-3_5 -->
<!-- cb-square-1_1--cb-square-2_1 -->
<!-- cb-square-1_2 -->
 
 cb-square-1_2 
 
 
<!-- cb-square-1_1--cb-square-1_2 -->
<!-- cb-square-1_2--cb-square-1_3 -->
<!-- cb-square-1_2--cb-square-2_2 -->
<!-- cb-square-0_3 -->
 
 cb-square-0_3 
 
 
<!-- cb-square-0_3--cb-square-1_3 -->
<!-- cb-square-0_3--cb-square-0_4 -->
<!-- cb-square-1_5--cb-square-2_5 -->
<!-- cb-square-1_0 -->
 
 cb-square-1_0 
 
 
<!-- cb-square-1_0--cb-square-1_1 -->
<!-- cb-square-2_0 -->
 
 cb-square-2_0 
 
 
<!-- cb-square-1_0--cb-square-2_0 -->
<!-- cb-square-3_3--cb-square-3_4 -->
<!-- cb-square-2_0--cb-square-3_0 -->
<!-- cb-square-2_0--cb-square-2_1 -->
<!-- cb-square-0_0 -->
 
 cb-square-0_0 
 
 
<!-- cb-square-0_0--cb-square-0_1 -->
<!-- cb-square-0_0--cb-square-1_0 -->
<!-- cb-square-3_2--cb-square-3_3 -->
<!-- cb-square-0_2--cb-square-1_2 -->
<!-- cb-square-0_2--cb-square-0_3 -->
<!-- cb-tick-4 -->
 
 cb-tick-4 
 
 4 
 
<!-- cb-tick-c -->
 
 cb-tick-c 
 
 c 
 
<!-- cb-tick-d -->
 
 cb-tick-d 
 
 d 
 
<!-- cb-tick-c--cb-tick-d -->
<!-- cb-tick-2 -->
 
 cb-tick-2 
 
 2 
 
<!-- cb-tick-3 -->
 
 cb-tick-3 
 
 3 
 
<!-- cb-tick-2--cb-tick-3 -->
<!-- cb-tick-b -->
 
 cb-tick-b 
 
 b 
 
<!-- cb-tick-b--cb-tick-c -->
<!-- cb-tick-e -->
 
 cb-tick-e 
 
 e 
 
<!-- cb-tick-f -->
 
 cb-tick-f 
 
 f 
 
<!-- cb-tick-e--cb-tick-f -->
<!-- cb-tick-3--cb-tick-4 -->
<!-- cb-tick-1 -->
 
 cb-tick-1 
 
 1 
 
<!-- cb-tick-1--cb-tick-2 -->
<!-- cb-tick-d--cb-tick-e -->
<!-- cb-tick-a -->
 
 cb-tick-a 
 
 a 
 
<!-- cb-tick-a--cb-tick-b -->

### How come I invested so much in Graphviz DOT support in `Graph`?

Interactive plotting with D3.js is fairly unreliable Raku-wise -- I can only use it in VSCode with Jupyter. 
It was working in web browsers with the old Jupyter notebook framework. But after JupyterLab was introduced my newest Jupyter-anything installations do not work with the > JavaScript settings for plotting.
(Making static HTML pages with D3.js plots is fine.)
So, finding a more robust and universal alternative was really needed. After some discussion with a Raku-enthusiast known as "timo"
I made finding such alternatives a priority. 
So, I looked for JavaScript alternatives to plot graphs first. I was also looking -- separately -- for JavaScript-based ways to use the Graphviz DOT language.
At some point I figured out that Graphviz layout engines are fairly install-able / deploy-able in many operating systems, so just using it to generate SVG, PNG, etc. is both fine and reliable. 


----

## Christmas themed

Let us finish this post with a topical example -- we make a graph that looks like a Christmas tree and we color it accordingly.

Here we create a (largish) triangular grid graph:

In [101]:
my $g = Graph::TriangularGrid.new(30, 30);

Graph(vertexes => 976, edges => 2760, directed => False)

Here we plot a smaller version of it:

In [102]:
#% html
my $gSmall = Graph::TriangularGrid.new(2,2);
$gSmall.dot(:$engine, :!node-labels, edge-thickness => 8, node-font-size => 30, node-width => 0.6, size => 4):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 5 -->
 
 5 
 
 
<!-- 7 -->
 
 7 
 
 
<!-- 5--7 -->
 
 5--7 
 
 
<!-- 8 -->
 
 8 
 
 
<!-- 4 -->
 
 4 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 6 -->
 
 6 
 
 
<!-- 4--6 -->
 
 4--6 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 3 -->
 
 3 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 3--6 -->
 
 3--6 
 
 
<!-- 0 -->
 
 0 
 
 
<!-- 2 -->
 
 2 
 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 1 -->
 
 1 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 2--5 -->
 
 2--5 
 
 
<!-- 2--4 -->
 
 2--4 
 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 1--3 -->
 
 1--3 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 6--8 -->
 
 6--8 
 
 
<!-- 6--7 -->
 
 6--7

Derive rectangular area boundaries:

In [103]:
my ($min-y, $max-y) = $g.vertex-coordinates.values.map(*.tail).Array.&{ (.min, .max)};

my $bottom-min-x = $g.vertex-coordinates.values.grep(*.tail == $min-y).map(*.head).min;
my $bottom-max-x = $g.vertex-coordinates.values.grep(*.tail == $min-y).map(*.head).max;

my $top-min-x = $g.vertex-coordinates.values.grep(*.tail == $max-y).map(*.head).min;
my $top-max-x = $g.vertex-coordinates.values.grep(*.tail == $max-y).map(*.head).max;

150.68842025849298

Top left node:

In [104]:
my $top-vertex = $g.vertex-coordinates.grep({ $_.value.head == $top-min-x && $_.value.tail == $max-y })».key.head

255

"Weak" diagonal equation:

In [105]:
my ($x1, $y1) = ($bottom-max-x, $min-y);
my ($x2, $y2) = |$g.vertex-coordinates{$top-vertex};

my $k = ($y1 - $y2)/($x1 - $x2);
my $n = -(($x2*$y1 - $x1*$y2)/($x1 - $x2));

sub y-diag(Numeric:D $x) { $k * $x +$n }

&y-diag

Get vertices under the "diagonal":

In [106]:
my @focus-vertexes = $g.vertex-coordinates.grep({  $_.value.tail ≤ y-diag($_.value.head) + 0.001 })».key;
@focus-vertexes.elems

499

Get the "pyramid" from the recursively obtained vertexes:

In [107]:
my $c-tree = $g.subgraph(@focus-vertexes); 

Graph(vertexes => 498, edges => 1395, directed => False)

Remove random parts:

In [108]:
my $c-tree2 = $c-tree.difference( $c-tree.neighborhood-graph($c-tree.vertex-list.pick(36).grep({ $_ ne $top-vertex }), d => 1) );

Graph(vertexes => 498, edges => 1012, directed => False)

Plot with highlights:

In [109]:
#%html
my @vs = $c-tree2.vertex-degree(:p).grep(*.value≤4)».key;

$c-tree2.dot( 
    highlight => {FloralWhite => @vs, Ivory => @vs.pick(12).List},
    node-width => 1.6,
    node-height => 1.6,
    node-fill-color => 'Red', 
    node-shape => 'circle',
    graph-size => 6,
    edge-color => 'Green',
    edge-width => 28,
    node-font-size => 380,
    pad => 2,
    node-labels => {$top-vertex => '⭐️', |(($g.vertex-list (-) $top-vertex).keys X=> '')},
    engine => 'neato'
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 444 -->
 
 444 
 
 
<!-- 475 -->
 
 475 
 
 
<!-- 444--475 -->
 
 444--475 
 
 
<!-- 460 -->
 
 460 
 
 
<!-- 444--460 -->
 
 444--460 
 
 
<!-- 17 -->
 
 17 
 
 
<!-- 26 -->
 
 26 
 
 
<!-- 17--26 -->
 
 17--26 
 
 
<!-- 22 -->
 
 22 
 
 
<!-- 17--22 -->
 
 17--22 
 
 
<!-- 21 -->
 
 21 
 
 
<!-- 17--21 -->
 
 17--21 
 
 
<!-- 690 -->
 
 690 
 
 
<!-- 705 -->
 
 705 
 
 
<!-- 690--705 -->
 
 690--705 
 
 
<!-- 50 -->
 
 50 
 
 
<!-- 58 -->
 
 58 
 
 
<!-- 50--58 -->
 
 50--58 
 
 
<!-- 65 -->
 
 65 
 
 
<!-- 50--65 -->
 
 50--65 
 
 
<!-- 57 -->
 
 57 
 
 
<!-- 50--57 -->
 
 50--57 
 
 
<!-- 229 -->
 
 229 
 
 
<!-- 244 -->
 
 244 
 
 
<!-- 229--244 -->
 
 229--244 
 
 
<!-- 245 -->
 
 245 
 
 
<!-- 229--245 -->
 
 229--245 
 
 
<!-- 260 -->
 
 260 
 
 
<!-- 229--260 -->
 
 229--260 
 
 
<!-- 123 -->
 
 123 
 
 
<!-- 146 -->
 
 146 
 
 
<!-- 123--146 -->
 
 123--146 
 
 
<!-- 135 -->
 
 135 
 
 
<!-- 123--135 -->
 
 123--135 
 
 
<!-- 134 -->
 
 134 
 
 
<!-- 123--134 -->
 
 123--134 
 
 
<!-- 505 -->
 
 505 
 
 
<!-- 520 -->
 
 520 
 
 
<!-- 505--520 -->
 
 505--520 
 
 
<!-- 284 -->
 
 284 
 
 
<!-- 67 -->
 
 67 
 
 
<!-- 75 -->
 
 75 
 
 
<!-- 67--75 -->
 
 67--75 
 
 
<!-- 76 -->
 
 76 
 
 
<!-- 67--76 -->
 
 67--76 
 
 
<!-- 84 -->
 
 84 
 
 
<!-- 67--84 -->
 
 67--84 
 
 
<!-- 416 -->
 
 416 
 
 
<!-- 432 -->
 
 432 
 
 
<!-- 416--432 -->
 
 416--432 
 
 
<!-- 447 -->
 
 447 
 
 
<!-- 416--447 -->
 
 416--447 
 
 
<!-- 431 -->
 
 431 
 
 
<!-- 416--431 -->
 
 416--431 
 
 
<!-- 492 -->
 
 492 
 
 
<!-- 508 -->
 
 508 
 
 
<!-- 492--508 -->
 
 492--508 
 
 
<!-- 523 -->
 
 523 
 
 
<!-- 492--523 -->
 
 492--523 
 
 
<!-- 507 -->
 
 507 
 
 
<!-- 492--507 -->
 
 492--507 
 
 
<!-- 34 -->
 
 34 
 
 
<!-- 46 -->
 
 46 
 
 
<!-- 34--46 -->
 
 34--46 
 
 
<!-- 39 -->
 
 39 
 
 
<!-- 34--39 -->
 
 34--39 
 
 
<!-- 40 -->
 
 40 
 
 
<!-- 34--40 -->
 
 34--40 
 
 
<!-- 352 -->
 
 352 
 
 
<!-- 368 -->
 
 368 
 
 
<!-- 352--368 -->
 
 352--368 
 
 
<!-- 383 -->
 
 383 
 
 
<!-- 352--383 -->
 
 352--383 
 
 
<!-- 367 -->
 
 367 
 
 
<!-- 352--367 -->
 
 352--367 
 
 
<!-- 278 -->
 
 278 
 
 
<!-- 293 -->
 
 293 
 
 
<!-- 278--293 -->
 
 278--293 
 
 
<!-- 309 -->
 
 309 
 
 
<!-- 278--309 -->
 
 278--309 
 
 
<!-- 294 -->
 
 294 
 
 
<!-- 278--294 -->
 
 278--294 
 
 
<!-- 567 -->
 
 567 
 
 
<!-- 583 -->
 
 583 
 
 
<!-- 567--583 -->
 
 567--583 
 
 
<!-- 161 -->
 
 161 
 
 
<!-- 173 -->
 
 173 
 
 
<!-- 161--173 -->
 
 161--173 
 
 
<!-- 187 -->
 
 187 
 
 
<!-- 161--187 -->
 
 161--187 
 
 
<!-- 174 -->
 
 174 
 
 
<!-- 161--174 -->
 
 161--174 
 
 
<!-- 185 -->
 
 185 
 
 
<!-- 213 -->
 
 213 
 
 
<!-- 185--213 -->
 
 185--213 
 
 
<!-- 198 -->
 
 198 
 
 
<!-- 185--198 -->
 
 185--198 
 
 
<!-- 199 -->
 
 199 
 
 
<!-- 185--199 -->
 
 185--199 
 
 
<!-- 253 -->
 
 253 
 
 
<!-- 130 -->
 
 130 
 
 
<!-- 261 -->
 
 261 
 
 
<!-- 292 -->
 
 292 
 
 
<!-- 261--292 -->
 
 261--292 
 
 
<!-- 277 -->
 
 277 
 
 
<!-- 261--277 -->
 
 261--277 
 
 
<!-- 276 -->
 
 276 
 
 
<!-- 261--276 -->
 
 261--276 
 
 
<!-- 196 -->
 
 196 
 
 
<!-- 210 -->
 
 210 
 
 
<!-- 196--210 -->
 
 196--210 
 
 
<!-- 225 -->
 
 225 
 
 
<!-- 196--225 -->
 
 196--225 
 
 
<!-- 211 -->
 
 211 
 
 
<!-- 196--211 -->
 
 196--211 
 
 
<!-- 413 -->
 
 413 
 
 
<!-- 429 -->
 
 429 
 
 
<!-- 413--429 -->
 
 413--429 
 
 
<!-- 344 -->
 
 344 
 
 
<!-- 375 -->
 
 375 
 
 
<!-- 344--375 -->
 
 344--375 
 
 
<!-- 360 -->
 
 360 
 
 
<!-- 344--360 -->
 
 344--360 
 
 
<!-- 359 -->
 
 359 
 
 
<!-- 344--359 -->
 
 344--359 
 
 
<!-- 101 -->
 
 101 
 
 
<!-- 125 -->
 
 125 
 
 
<!-- 148 -->
 
 148 
 
 
<!-- 125--148 -->
 
 125--148 
 
 
<!-- 136 -->
 
 136 
 
 
<!-- 125--136 -->
 
 125--136 
 
 
<!-- 720 -->
 
 720 
 
 
<!-- 705